Inspiration 🥷
https://github.com/ZA3karia/PDF2TEXT/blob/master/ebook_to_text.ipynb

In [1]:
!pip install ebooklib
!pip install BeautifulSoup4
!pip install openai

In [2]:
# conda env nlp
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

In [3]:
def epub2thtml(epub_path):
    book = epub.read_epub(epub_path)
    chapters = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters.append(item.get_content())
    return chapters

In [4]:
blacklist = [
	'[document]',
	'noscript',
	'header',
	'html',
	'meta',
	'head', 
	'input',
	'script',
	# there may be more elements you don't want, such as "style", etc.
]
def chap2text(chap):
    output = ''
    soup = BeautifulSoup(chap, 'html.parser')
    text = soup.find_all(text=True)
    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    return output


In [5]:
def thtml2ttext(thtml):
    Output = []
    for html in thtml:
        text =  chap2text(html)
        Output.append(text)
    return Output

In [6]:
def epub2text(epub_path):
    chapters = epub2thtml(epub_path)
    ttext = thtml2ttext(chapters)
    return ttext

In [7]:
out = epub2text('books/Great Expectations by Charles Dickens.epub')

/opt/homebrew/Caskroom/miniforge/base/envs/nlp/lib/python3.9/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [8]:
total_cost = 0
for i,c in enumerate(out):
    # Only transcribe chapters with more than 5000 characters
    if len(c) > 5000:
        # price is 0.015$ per 1000 characters
        print(i, len(c), c[:20].replace('\n', ''), f'cost {len(c)/1000*0.015:.2f}$')
        total_cost += len(c)/1000*0.015
print(f'total cost: {total_cost:.2f}$')

1 9781   Chapter I.  M y cost 0.15$
2 18482   Chapter II.  M  cost 0.28$
3 10314   Chapter III.  I cost 0.15$
4 18778   Chapter IV.  I  cost 0.28$
5 22642   Chapter V.  T h cost 0.34$
7 21763   Chapter VII.  A cost 0.33$
8 25511   Chapter VIII.   cost 0.38$
9 14501   Chapter IX.  W  cost 0.22$
10 13761   Chapter X.  T h cost 0.21$
11 29087   Chapter XI.  A  cost 0.44$
12 12226   Chapter XII.  M cost 0.18$
13 15778   Chapter XIII.   cost 0.24$
15 23356   Chapter XV.  A  cost 0.35$
16 10613   Chapter XVI.  W cost 0.16$
17 17971   Chapter XVII.   cost 0.27$
18 27634   Chapter XVIII.  cost 0.41$
19 30946   Chapter XIX.  M cost 0.46$
20 17249   Chapter XX.  T  cost 0.26$
21 9642   Chapter XXI.  C cost 0.14$
22 27325   Chapter XXII.   cost 0.41$
23 17849   Chapter XXIII.  cost 0.27$
24 12774   Chapter XXIV.   cost 0.19$
25 16055   Chapter XXV.  B cost 0.24$
26 15518   Chapter XXVI.   cost 0.23$
27 16507   Chapter XXVII.  cost 0.25$
28 13337   Chapter XXVIII.  cost 0.20$
29 26536   Chapter XX

In [ ]:
from openai import OpenAI

import os
os.environ["OPENAI_API_KEY"] = "<your key>"
client = OpenAI()

index = 0
for i,c in enumerate(out):
    # Only transcribe chapters with more than 5000 characters
    if len(c) > 5000:
      chunk = ''
      chunks = []
      for t in c.split('\n'):
          if len(chunk) + len(t) < 4000:
              chunk += t + '\n'
          else:
              chunks.append(chunk)
              chunk = t + '\n'
      for i,payload in enumerate(chunks):
          response = client.audio.speech.create(
              model="tts-1",
              voice="onyx",
              input=payload,
            )  
          response.stream_to_file(f'{index:03} {c[:20]}_{i}.mp3')
          index += 1